In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1"

In [2]:
import math
import transformers
from datasets import load_dataset
from pathlib import Path
from transformers import AutoTokenizer,TrainingArguments,Trainer

# Fine-tuning a language model

## Preparing the dataset

For each of those tasks, we will use the [Wikitext 2]() dataset as an example. You can load it very easily with the 🤗 Datasets library.

In [3]:
files = list(map(str, Path("data/wiki-20220301-en").glob("*.parquet")))
datasets = load_dataset("parquet", data_files=files, split="train")

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

In [4]:
datasets[10]

{'id': '63665707',
 'url': 'https://en.wikipedia.org/wiki/Sardiha%20railway%20station',
 'title': 'Sardiha railway station',
 'text': 'Sardiha railway station is a railway station on Howrah–Nagpur–Mumbai line under Kharagpur railway division of South Eastern Railway zone. It is situated at Dhadkinala in Jhargram district in the Indian state of West Bengal. It is  from Kharagpur Junction.\n\nReferences\n\nRailway stations in Jhargram district\nKharagpur railway division\nJhargram district'}

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [5]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [6]:
show_random_elements(datasets)

As we can see, some of the texts are a full paragraph of a Wikipedia article while others are just titles or empty lines.

## Masked language modeling

For masked language modeling (MLM) we are going to use the same preprocessing as before for our dataset with one additional step: we will randomly mask some tokens (by replacing them by `[MASK]`) and the labels will be adjusted to only include the masked tokens (we don't have to predict the non-masked tokens).

We will use the [`distilroberta-base`](https://huggingface.co/distilroberta-base) model for this example. You can pick any of the checkpoints listed [here](https://huggingface.co/models?filter=masked-lm) instead:

In [7]:
model_checkpoint = "model/deberta-v3-large-hf-weights"

We can apply the same tokenization function as before, we just need to update our tokenizer to use the checkpoint we just picked:

In [8]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=12, remove_columns=["text", "url", "title"])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/zhangdonghui/anaconda3/envs/kaggle/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map (num_proc=12):   0%|          | 0/6458670 [00:00<?, ? examples/s]

In [ ]:
for k in next(iter(tokenized_datasets)).keys():
    print(k)

In [ ]:
block_size = 128

In [ ]:
def group_texts(examples):
    """将一组文本实例重新组合成一个适合训练的数据格式"""
    # Concatenate all texts.
    concatenated_examples = {k: []for k in examples.keys()}
    for k in examples.keys():
        for text in examples[k]:
            concatenated_examples[k].extend(text)
    # concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=12,
)

The rest is very similar to what we had, with two exceptions. First we use a model suitable for masked LM:

In [ ]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

We redefine our `TrainingArguments`:

In [ ]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wiki-sci",
    evaluation_strategy = "epoch",
    warmup_ratio=0.1,
    learning_rate=50e-7,
    weight_decay=0.01,
    bf16=True,
    save_total_limit=1,
    save_strategy="epoch",
    auto_find_batch_size=True,
    num_train_epochs=20,
    load_best_model_at_end=True,
    per_device_train_batch_size=512,
    output_dir="./save_checkpoints"
)

Finally, we use a special `data_collator`. The `data_collator` is a function that is responsible of taking the samples and batching them in tensors. In the previous example, we had nothing special to do, so we just used the default for this argument. Here we want to do the random-masking. We could do it as a pre-processing step (like the tokenization) but then the tokens would always be masked the same way at each epoch. By doing this step inside the `data_collator`, we ensure this random masking is done in a new way each time we go over the data.

To do this masking for us, the library provides a `DataCollatorForLanguageModeling`. We can adjust the probability of the masking:

In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

Then we just have to pass everything to `Trainer` and begin training:

In [ ]:
train_testvalid = lm_datasets.train_test_split(test_size=0.05)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_testvalid["train"],
    eval_dataset=train_testvalid["test"],
    data_collator=data_collator,
)

In [ ]:
trainer.train()

Like before, we can evaluate our model on the validation set. The perplexity is much lower than for the CLM objective because for the MLM objective, we only have to make predictions for the masked tokens (which represent 15% of the total here) while having access to the rest of the tokens. It's thus an easier task for the model.

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

You can now upload the result of the training to the Hub, just execute this instruction: